# Import libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Read dataset

In [2]:
diamonds_test = pd.read_csv('data/diamonds.csv')
diamonds_rick = pd.read_csv('data/rick_diamonds.csv')

# Data Cleaning 

# Exploratory Analysis

# Data transformation

# Modeling

In [4]:
previsao = diamonds_test['price'].mean()
diamonds_rick['price_predicted'] = previsao
diamonds_rick.to_csv('testes_price/price_mean', index=False)

# Results


In [ ]:
# erro usando a média foi: 3980.71
